In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-addon-mpf

*Tinatayang paggamit ng QPU: Apat na minuto sa isang Heron r2 processor (TANDAAN: Tantiya lamang ito. Maaaring mag-iba ang inyong runtime.)*

## Pangkalahatang-ideya

Ipinapakita ng tutorial na ito kung paano gamitin ang isang Multi-Product Formula (MPF) upang makamit ang mas mababang Trotter error sa ating observable kumpara sa error na maidudulot ng pinakamalalim na Trotter circuit na ating isasagawa.
Binabawasan ng mga MPF ang Trotter error ng Hamiltonian dynamics sa pamamagitan ng may-timbang na kombinasyon ng ilang circuit execution. Isaalang-alang ang gawain ng paghahanap ng mga inaasahang halaga ng observable para sa quantum state
$\rho(t)=e^{-i H t} \rho(0) e^{i H t}$ na may Hamiltonian na $H$. Maaaring gumamit ng mga Product Formula (PF) upang tantiyahin ang time-evolution na $e^{-i H t}$ sa pamamagitan ng sumusunod:

- Isulat ang Hamiltonian na $H$ bilang $H=\sum_{a=1}^d F_a,$ kung saan ang $F_a$ ay mga Hermitian operator na ang bawat katumbas na unitary ay maaaring mahusay na maipatupad sa isang quantum device.
- Tantiyahin ang mga terminong $F_a$ na hindi nag-commute sa isa't isa.

Kaya naman, ang first-order PF (Lie-Trotter formula) ay:

$$S_1(t):=\prod_{a=1}^d e^{-i F_a t},$$

na may quadratic error term na $S_1(t)=e^{-i H t}+\mathcal{O}\left(t^{2}\right)$. Maaari ring gumamit ng higher-order PF (Lie-Trotter-Suzuki formula), na mas mabilis na magkakalapit, at tinukoy nang rekursibo bilang:

$$S_2(t):=\prod_{a=1}^d e^{-i F_a t/2}\prod_{a=1}^d e^{-i F_a t/2}$$

$$S_{2 \chi}(t):= S_{2 \chi -2}(s_{\chi}t)^2 S_{2 \chi -2}((1-4s_{\chi})t)S_{2 \chi -2}(s_{\chi}t)^2,$$

kung saan ang $\chi$ ay ang order ng symmetric PF at $s_p = \left( 4 - 4^{1/(2p-1)} \right)^{-1}$. Para sa matagalang time-evolution, maaaring hatiin ang time interval na $t$ sa $k$ na interval, na tinatawag na mga Trotter step, na may tagal na $t/k$ at tantiyahin ang time-evolution sa bawat interval gamit ang $\chi$ order product formula na $S_{\chi}$. Kaya naman, ang PF ng order na $\chi$ para sa time-evolution operator sa $k$ Trotter step ay:

$$ S_{\chi}^{k}(t) = \left[ S_{\chi} \left( \frac{t}{k} \right)\right]^k = e^{-i H t}+O\left(t \left( \frac{t}{k} \right)^{\chi} \right)$$

kung saan ang error term ay bumababa kasabay ng bilang ng mga Trotter step na $k$ at ng order na $\chi$ ng PF.

Kapag may ibinigay na integer na $k \geq 1$ at isang product formula na $S_{\chi}(t)$, ang tinantyang time-evolved state na $\rho_k(t)$ ay maaaring makuha mula sa $\rho_0$ sa pamamagitan ng pag-apply ng $k$ ulit ng product formula na $S_{\chi}\left(\frac{t}{k}\right)$.

$$
\rho_k(t)=S_{\chi}\left(\frac{t}{k}\right)^k \rho_0 S_{\chi}\left(\frac{t}{k}\right)^{-k}
$$

Ang $\rho_k(t)$ ay isang tantiya para sa $\rho(t)$ na may Trotter approximation error na ||$\rho_k(t)-\rho(t) ||$. Kung isasaalang-alang natin ang isang linear na kombinasyon ng mga Trotter approximation ng $\rho(t)$:

$$
\mu(t) = \sum_{j}^{l} x_j \rho^{k_j}_{j}\left(\frac{t}{k_j}\right) + \text{some remaining Trotter error} \, ,
$$

kung saan ang $x_j$ ay ang ating mga weighting coefficient, ang $\rho^{k_j}_j$ ay ang density matrix na katumbas ng pure state na nakuha sa pamamagitan ng pag-evolve ng initial state gamit ang product formula, $S^{k_j}_{\chi}$, na nagsasangkot ng $k_j$ Trotter step, at ang $j \in {1, ..., l}$ ay nag-i-index ng bilang ng mga PF na bumubuo sa MPF. Lahat ng termino sa $\mu(t)$ ay gumagamit ng parehong product formula na $S_{\chi}(t)$ bilang basehan.
Ang layunin ay mapabuti ang ||$\rho_k(t)-\rho(t) \|$ sa pamamagitan ng paghahanap ng $\mu(t)$ na may mas mababang $\|\mu(t)-\rho(t)\|$.

* Hindi kailangang maging pisikal na estado ang $\mu(t)$ dahil hindi kailangang positibo ang $x_i$. Ang layunin dito ay mabawasan ang error sa inaasahang halaga ng mga observable at hindi upang makahanap ng pisikal na kapalit para sa $\rho(t)$.
* Tinutukoy ng $k_j$ ang parehong circuit depth at antas ng Trotter approximation. Ang mas maliit na halaga ng $k_j$ ay nagbubunga ng mas maikling mga circuit, na nagdudulot ng mas kaunting circuit error ngunit magiging mas hindi tumpak na tantiya ng nais na estado.

Ang susi dito ay ang natitirang Trotter error na ibinibigay ng $\mu(t)$ ay mas maliit kaysa sa Trotter error na makukuha kung gagamitin lamang ang pinakamataas na halaga ng $k_j$.

Maaaring tingnan ang kapaki-pakinabang nito mula sa dalawang perspektibo:

1. Para sa isang nakapirming badyet ng Trotter step na maisasagawa, makakakuha ng mga resulta na may mas maliit na Trotter error sa kabuuan.
2. Kung may target na bilang ng Trotter step na masyadong malaki upang maisagawa, maaaring gamitin ang MPF upang makahanap ng isang koleksyon ng mas mababang-lalim na mga circuit na nagbubunga ng katulad na Trotter error.

## Mga Kinakailangan

Bago simulan ang tutorial na ito, tiyakin na naka-install ang mga sumusunod:

* Qiskit SDK v1.0 o mas bago, na may suporta sa [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime v0.22 o mas bago (`pip install qiskit-ibm-runtime`)
* MPF Qiskit addons (`pip install qiskit_addon_mpf`)
* Qiskit addons utils (`pip install qiskit_addon_utils`)
* Quimb library (`pip install quimb`)
* Qiskit Quimb library (`pip install qiskit-quimb`)
* Numpy v0.21 para sa compatibility sa iba't ibang package (`pip install numpy==0.21`)

## Bahagi I. Maliit na halimbawa

### Suriin ang katatagan ng MPF

Walang malinaw na limitasyon sa pagpili ng bilang ng mga Trotter step na $k_j$ na bumubuo sa MPF state na $\mu(t)$. Gayunpaman, dapat itong maingat na piliin upang maiwasan ang kawalang-tatag sa mga inaasahang halagang kinakalkula mula sa $\mu(t)$. Ang isang magandang pangkalahatang tuntunin ay itakda ang pinakamaliit na Trotter step na $k_{\text{min}}$ upang $t/k_{\text{min}} \lt 1$. Kung nais ninyong matuto pa tungkol dito at kung paano pipiliin ang inyong iba pang $k_j$ na halaga, sumangguni sa gabay na [How to choose the Trotter steps for an MPF](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html).

Sa halimbawa sa ibaba, sinusuri natin ang katatagan ng MPF solution sa pamamagitan ng pagkalkula ng inaasahang halaga ng magnetisasyon para sa isang hanay ng mga oras gamit ang iba't ibang time-evolved state. Partikular, inihahambing natin ang mga inaasahang halagang kinakalkula mula sa bawat isa sa mga tinantyang time-evolution na isinagawa gamit ang katumbas na Trotter step at ang iba't ibang MPF model (static at dynamic na coefficient) kasama ang eksaktong halaga ng time-evolved observable. Una, tukuyin natin ang mga parameter para sa mga Trotter formula at ang mga oras ng evolution.

In [1]:
import numpy as np

mpf_trotter_steps = [1, 2, 4]
order = 2
symmetric = False

trotter_times = np.arange(0.5, 1.55, 0.1)
exact_evolution_times = np.arange(trotter_times[0], 1.55, 0.05)

Para sa halimbawang ito, gagamitin natin ang Neel state bilang initial state na $\vert \text{Neel} \rangle = \vert 0101...01 \rangle$ at ang Heisenberg model sa isang linya ng 10 site para sa Hamiltonian na namamahala sa time-evolution

$$
\hat{\mathcal{H}}_{Heis} = J \sum_{i=1}^{L-1} \left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ Z_i Z_{(i+1)} \right) \, ,
$$

kung saan ang $J$ ay ang coupling strength para sa mga pinakamalapit na gilid.

In [2]:
from qiskit.transpiler import CouplingMap
from rustworkx.visualization import graphviz_draw
from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian
import numpy as np


L = 10

# Generate some coupling map to use for this example
coupling_map = CouplingMap.from_line(L, bidirectional=False)
graphviz_draw(coupling_map.graph, method="circo")

# Get a qubit operator describing the Heisenberg field model
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(1.0, 1.0, 1.0),
    ext_magnetic_field=(0.0, 0.0, 0.0),
)


print(hamiltonian)

SparsePauliOp(['IIIIIIIXXI', 'IIIIIIIYYI', 'IIIIIIIZZI', 'IIIIIXXIII', 'IIIIIYYIII', 'IIIIIZZIII', 'IIIXXIIIII', 'IIIYYIIIII', 'IIIZZIIIII', 'IXXIIIIIII', 'IYYIIIIIII', 'IZZIIIIIII', 'IIIIIIIIXX', 'IIIIIIIIYY', 'IIIIIIIIZZ', 'IIIIIIXXII', 'IIIIIIYYII', 'IIIIIIZZII', 'IIIIXXIIII', 'IIIIYYIIII', 'IIIIZZIIII', 'IIXXIIIIII', 'IIYYIIIIII', 'IIZZIIIIII', 'XXIIIIIIII', 'YYIIIIIIII', 'ZZIIIIIIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


The observable that we will be measuring is magnetization on a pair of qubits in the middle of the chain.

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp.from_sparse_list(
    [("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L
)
print(observable)

SparsePauliOp(['IIIIZZIIII'],
              coeffs=[1.+0.j])


Ang observable na ating susukat ay ang magnetisasyon sa isang pares ng qubit sa gitna ng chain.

In [4]:
from qiskit.circuit.library import XXPlusYYGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.optimization.collect_and_collapse import (
    CollectAndCollapse,
    collect_using_filter_function,
    collapse_to_operation,
)
from functools import partial


def filter_function(node):
    return node.op.name in {"rxx", "ryy"}


collect_function = partial(
    collect_using_filter_function,
    filter_function=filter_function,
    split_blocks=True,
    min_block_size=1,
)


def collapse_to_xx_plus_yy(block):
    param = 0.0
    for node in block.data:
        param += node.operation.params[0]
    return XXPlusYYGate(param)


collapse_function = partial(
    collapse_to_operation,
    collapse_function=collapse_to_xx_plus_yy,
)

pm = PassManager()
pm.append(CollectAndCollapse(collect_function, collapse_function))

Then we create the circuits implementing the approximate Trotter time-evolutions.

In [5]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


# Initial Neel state preparation
initial_state_circ = QuantumCircuit(L)
initial_state_circ.x([i for i in range(L) if i % 2 != 0])


all_circs = []
for total_time in trotter_times:
    mpf_trotter_circs = [
        generate_time_evolution_circuit(
            hamiltonian,
            time=total_time,
            synthesis=SuzukiTrotter(reps=num_steps, order=order),
        )
        for num_steps in mpf_trotter_steps
    ]

    mpf_trotter_circs = pm.run(
        mpf_trotter_circs
    )  # Collect XX and YY into XX + YY

    mpf_circuits = [
        initial_state_circ.compose(circuit) for circuit in mpf_trotter_circs
    ]
    all_circs.append(mpf_circuits)

In [6]:
mpf_circuits[-1].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/92dc20a7-0.avif" alt="Output of the previous code cell" />

Pagkatapos ay lilikha tayo ng mga circuit na nagpapatupad ng mga tinantyang Trotter time-evolution.

In [24]:
from copy import deepcopy
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


aer_sim = AerSimulator()
estimator = Estimator(mode=aer_sim)

mpf_expvals_all_times, mpf_stds_all_times = [], []
for t, mpf_circuits in zip(trotter_times, all_circs):
    mpf_expvals = []
    circuits = [deepcopy(circuit) for circuit in mpf_circuits]
    pm_sim = generate_preset_pass_manager(
        backend=aer_sim, optimization_level=3
    )
    isa_circuits = pm_sim.run(circuits)
    result = estimator.run(
        [(circuit, observable) for circuit in isa_circuits], precision=0.005
    ).result()
    mpf_expvals = [res.data.evs for res in result]
    mpf_stds = [res.data.stds for res in result]
    mpf_expvals_all_times.append(mpf_expvals)
    mpf_stds_all_times.append(mpf_stds)

We also calculate the exact expectation values for comparison.

In [ ]:
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

exact_expvals = []
for t in exact_evolution_times:
    # Exact expectation values
    exp_H = expm(-1j * t * hamiltonian.to_matrix())
    initial_state = Statevector(initial_state_circ).data
    time_evolved_state = exp_H @ initial_state

    exact_obs = (
        time_evolved_state.conj()
        @ observable.to_matrix()
        @ time_evolved_state
    ).real
    exact_expvals.append(exact_obs)

![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/92dc20a7-0.avif)

Susunod, kinakalkula natin ang mga time-evolved na inaasahang halaga mula sa mga Trotter circuit.

In [9]:
from qiskit_addon_mpf.static import setup_static_lse

lse = setup_static_lse(mpf_trotter_steps, order=order, symmetric=symmetric)

Kinakalkula rin natin ang mga eksaktong inaasahang halaga para sa paghahambing.

In [10]:
lse.A

array([[1.      , 1.      , 1.      ],
       [1.      , 0.25    , 0.0625  ],
       [1.      , 0.125   , 0.015625]])

#### Mga static na MPF coefficient
Ang mga static MPF ay ang mga kung saan ang mga halaga ng $x_j$ ay hindi nakasalalay sa oras ng evolution na $t$. Isaalang-alang ang order na $\chi = 1$ PF na may $k_j$ Trotter step, maaari itong isulat bilang:

$$ S_1^{k_j}\left( \frac{t}{k_j} \right)=e^{-i H t}+ \sum_{n=1}^{\infty} A_n \frac{t^{n+1}}{k_j^n}  $$

kung saan ang $A_n$ ay mga matrix na nakasalalay sa mga commutator ng mga $F_a$ na termino sa decomposition ng Hamiltonian. Mahalaga pong tandaan na ang $A_n$ mismo ay independyente sa oras at sa bilang ng mga Trotter step na $k_j$. Kaya naman, posibleng kanselahin ang mga lower-order error term na nag-aambag sa $\mu(t)$ sa pamamagitan ng maingat na pagpili ng mga timbang na $x_j$ ng linear na kombinasyon. Upang kanselahin ang Trotter error para sa unang $l-1$ na termino (ang mga ito ay magbibigay ng pinakamalaking kontribusyon dahil katumbas ang mga ito ng mas mababang bilang ng Trotter step) sa expression para sa $\mu(t)$, ang mga coefficient na $x_j$ ay dapat matugunan ang mga sumusunod na equation:

$$ \sum_{j=1}^l x_j = 1 $$
$$ \sum_{j=1}^{l-1} \frac{x_j}{k_j^{n}} = 0 $$

na may $n=0, ... l-2$. Tinitiyak ng unang equation na walang bias sa constructed state na $\mu(t)$, habang tinitiyak ng pangalawang equation ang pagkansela ng mga Trotter error. Para sa higher-order PF, ang pangalawang equation ay nagiging $ \sum_{j=1}^{l-1} \frac{x_j}{k_j^{\eta}} = 0 $ kung saan ang $\eta = \chi + 2n$ para sa mga symmetric PF at $\eta = \chi + n$ kung hindi, na may $n=0, ..., l-2$. Ang resulta na error (Refs. [\[1\]](#references),[\[2\]](#references)) ay

$$ \epsilon = \mathcal{O} \left( \frac{t^{l+1}}{k_1^l} \right).$$

Ang pagtatukoy ng mga static MPF coefficient para sa isang ibinigay na hanay ng mga halaga ng $k_j$ ay katumbas ng paglutas ng linear system ng mga equation na tinukoy ng dalawang equation sa itaas para sa mga variable na $x_j$: $Ax=b$. Kung saan ang $x$ ang ating mga coefficient ng interes, ang $A$ ay isang matrix na nakasalalay sa $k_j$ at sa uri ng PF na ating ginagamit ($S$), at ang $b$ ay isang vector ng mga constraint. Partikular:

$$A_{0,j} = 1 $$
$$A_{i>0,j} = k_{j}^{-(\chi + s(i-1))}$$
$$b_0 = 1$$
$$b_{i>0} = 0 $$

kung saan ang $\chi$ ay ang ``order``, ang $s$ ay $2$ kung ang ``symmetric`` ay ``True`` at $1$ kung hindi, ang $k_{j}$ ay ang ``trotter_steps``, at ang $x$ ay ang mga variable na lulutas. Ang mga indeks na $i$ at $j$ ay nagsisimula sa $0$. Maaari rin nating visualisahin ito sa matrix form:

$$
A =
\begin{bmatrix}
A_{0,0} & A_{0,1} & A_{0,2} & ... \\
A_{1,0} & A_{1,1} & A_{1,2} & ...  \\
A_{2,0} & A_{2,1} & A_{2,2} & ...  \\
... & ... & ... & ...
\end{bmatrix} =
\begin{bmatrix}
1 & 1 & 1 & ... \\
k_{0}^{-(\chi + s(1-1))} & k_{1}^{-(\chi + s(1-1))} & k_{2}^{-(\chi + s(1-1))} & ... \\
k_{0}^{-(\chi + s(2-1))} & k_{1}^{-(\chi + s(2-1))} & k_{2}^{-(\chi + s(2-1))} & ... \\
... & ... & ... & ...
\end{bmatrix}
$$

at

$$
b =
\begin{bmatrix}
b_{0} \\
b_{1} \\
b_{2}  \\
...
\end{bmatrix} =
\begin{bmatrix}
1 \\
0 \\
0  \\
...
\end{bmatrix}
$$

Para sa karagdagang detalye, sumangguni sa dokumentasyon ng Linear System of Equations ([LSE](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.LSE.html)).

Maaari tayong makahanap ng solusyon para sa $x$ nang analitiko bilang $x = A^{-1}b$; tingnan halimbawa ang Refs. [\[1\]](#references) o [\[2\]](#references).
Gayunpaman, ang eksaktong solusyong ito ay maaaring "ill-conditioned", na nagbubunga ng napakalaking L1-norm ng ating mga coefficient na $x$, na maaaring magdulot ng masamang pagganap ng MPF.
Sa halip, maaari ring makakuha ng tinantyang solusyon na nagpapaliit ng L1-norm ng $x$ upang subukang i-optimize ang gawi ng MPF.
##### I-set up ang LSE
Ngayon na napili na natin ang ating mga halaga ng $k_j$, kailangan muna nating itayo ang LSE, $Ax=b$ gaya ng ipinaliwanag sa itaas.
Ang matrix na $A$ ay nakasalalay hindi lamang sa $k_j$ kundi pati na rin sa ating pagpili ng PF, partikular ang _order_ nito.
Bukod pa rito, maaari rin ninyong isaalang-alang kung ang PF ay symmetric o hindi (tingnan ang [\[1\]](#references)) sa pamamagitan ng pagtatakda ng `symmetric=True/False`.
Gayunpaman, hindi ito kinakailangan, gaya ng ipinapakita ng Ref. [\[2\]](#references).

In [11]:
lse.b

array([1., 0., 0.])

Suriin natin ang mga halagang pinili sa itaas upang itayo ang matrix na $A$ at ang vector na $b$. Sa $j=0,1, 2$ Trotter step na $k_j = [1, 2, 4]$, order na $\chi = 2$ at pagpili ng non-symmetric Trotter step ($s=1$), makikita natin na ang mga matrix element ng $A$ sa ibaba ng unang row ay tinutukoy ng expression na $A_{i>0,j} = k_{j}^{-(2 + 1(i-1))}$, partikular:

$$ A_{0,0} = A_{0,1} = A_{0,2} =  1 $$
$$ A_{1,j} = k_{j}^{-1}  \rightarrow A_{1,0} = \frac{1}{1^2}, \;, A_{1,1} = \frac{1}{2^2}, \;, A_{1,2} = \frac{1}{4^2}$$
$$ A_{2,j} = k_{j}^{-2}  \rightarrow A_{2,0} = \frac{1}{1^3}, \;, A_{2,1} = \frac{1}{2^3}, \;, A_{2,2} = \frac{1}{4^3}$$

o sa matrix form:

$$
A =
\begin{bmatrix}
1 & 1 & 1\\
1 & \frac{1}{2^2} & \frac{1}{4^2}  \\
1 & \frac{1}{2^3} & \frac{1}{4^3}  \\
\end{bmatrix}
$$

Maaari itong makita sa pamamagitan ng pagsusuri ng `lse` object:

In [12]:
mpf_coeffs = lse.solve()
print(
    f"The static coefficients associated with the ansatze are: {mpf_coeffs}"
)

The static coefficients associated with the ansatze are: [ 0.04761905 -0.57142857  1.52380952]


##### Optimize for $x$ using an exact model

Alternatively to computing $x=A^{-1}b$, you can also use [setup_exact_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_exact_model.html) to construct a [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) instance that uses the LSE as constraints and whose optimal solution will yield $x$.

In the next section, it will be clear why this interface exists.

In [13]:
from qiskit_addon_mpf.costs import setup_exact_problem

model_exact, coeffs_exact = setup_exact_problem(lse)
model_exact.solve()
print(coeffs_exact.value)

[ 0.04761905 -0.57142857  1.52380952]


Habang ang vector ng mga constraint na $b$ ay may mga sumusunod na elemento:
$$ b_{0} = 1 $$
$$ b_1 = b_2 = 0 $$

Kaya naman,

$$
b =
\begin{bmatrix}
1 \\
0 \\
0
\end{bmatrix}
$$

At katulad nito sa `lse`:

In [14]:
print(
    "L1 norm of the exact coefficients:",
    np.linalg.norm(coeffs_exact.value, ord=1),
)  # ord specifies the norm. ord=1 is for L1

L1 norm of the exact coefficients: 2.1428571428556378


##### Optimize for $x$ using an approximate model

It might happen that the L1 norm for the chosen set of $k_j$ values is deemed too high.
If that is the case and you cannot choose a different set of $k_j$ values, you can use an approximate solution to the LSE instead of an exact one.

To do so, simply use [setup_approximate_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_approximate_model.html) to construct a different [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) instance, which constrains the L1-norm to a chosen threshold while minimizing the difference of $Ax$ and $b$.

In [16]:
from qiskit_addon_mpf.costs import setup_sum_of_squares_problem

model_approx, coeffs_approx = setup_sum_of_squares_problem(
    lse, max_l1_norm=1.5
)
model_approx.solve()
print(coeffs_approx.value)
print(
    "L1 norm of the approximate coefficients:",
    np.linalg.norm(coeffs_approx.value, ord=1),
)

[-1.10294118e-03 -2.48897059e-01  1.25000000e+00]
L1 norm of the approximate coefficients: 1.5


Ang `lse` object ay may mga pamamaraan para sa paghahanap ng mga static na coefficient na $x_j$ na natutugunan ang sistema ng mga equation.

In [17]:
from qiskit_addon_utils.slicing import slice_by_depth
from qiskit_addon_mpf.backends.tenpy_layers import LayerModel
from qiskit_addon_mpf.backends.tenpy_layers import LayerwiseEvolver
from functools import partial

# Create approximate time-evolution circuits
single_2nd_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=order)
)
single_2nd_order_circ = pm.run(single_2nd_order_circ)  # collect XX and YY

# Find layers in the circuit
layers = slice_by_depth(single_2nd_order_circ, max_slice_depth=1)

# Create tensor network models
models = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz") for layer in layers
]

# Create the time-evolution object
approx_factory = partial(
    LayerwiseEvolver,
    layers=models,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 2,
    },
)

<Admonition type="warning">
The options of `LayerwiseEvolver` that determine the details of the tensor network simulation must be chosen carefully to avoid setting up an ill-defined optimization problem.
</Admonition>

We then set up the exact evolver (for example, [`ExactEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ExactEvolverFactory)), which returns an [`Evolver`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.html#qiskit_addon_mpf.backends.Evolver) object computing the true or “reference” time-evolution. Realistically, we would approximate the exact evolution by using a higher-order Suzuki–Trotter formula or another reliable method with a small time step. Below, we approximate the exact time-evolved state with a fourth-order Suzuki-Trotter formula using a small time step `dt=0.1`, which means the number of Trotter steps used at time $t$ is $k=t/dt$. We also specify some TeNPy-specific truncation options to bound the maximum bond dimension of the underlying tensor network, as well as the minimum singular values of the split tensor network bonds. These parameters can affect the accuracy of the expectation value calculated with the dynamic MPF coefficients, so it is important to explore a range of values to find the optimal balance between computational time and accuracy. Note that the calculation of the MPF coefficients does not rely of the expectation value of the PF obtained from hardware execution, and therefore it can be tuned in post-processing.

In [20]:
single_4th_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=4)
)
single_4th_order_circ = pm.run(single_4th_order_circ)
exact_model_layers = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz")
    for layer in slice_by_depth(single_4th_order_circ, max_slice_depth=1)
]

exact_factory = partial(
    LayerwiseEvolver,
    layers=exact_model_layers,
    dt=0.1,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 2,
    },
)

##### I-optimize ang $x$ gamit ang isang eksaktong modelo
Bilang alternatibo sa pagkalkula ng $x=A^{-1}b$, maaari rin ninyong gamitin ang [setup_exact_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_exact_model.html) upang itayo ang isang [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) instance na gumagamit ng LSE bilang mga constraint at ang optimal na solusyon nito ay magbubunga ng $x$.

Sa susunod na seksyon, magiging malinaw kung bakit umiiral ang interface na ito.

In [ ]:
from qiskit_addon_mpf.backends.tenpy_tebd import MPOState
from qiskit_addon_mpf.backends.tenpy_tebd import MPS_neel_state


def identity_factory():
    return MPOState.initialize_from_lattice(models[0].lat, conserve=True)


mps_initial_state = MPS_neel_state(models[0].lat)

For each time step $t$ we set up the dynamic linear system of equations with the [`setup_dynamic_lse`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html) method. The corresponding object contains the information about the dynamic MPF problem: `lse.A` gives the Gram matrix $M$ while `lse.b` gives the overlap $L$. We can then solve the LSE (when not ill-defined) to find the dynamic coefficients using the `setup_frobenius_problem`. It is important to note the difference with the static coefficients, which only depend on the details of the product formula used and are independent of the details of the time-evolution (Hamiltonian and initial state).

In [22]:
from qiskit_addon_mpf.dynamic import setup_dynamic_lse
from qiskit_addon_mpf.costs import setup_frobenius_problem

mpf_dynamic_coeffs_list = []
for t in trotter_times:
    print(f"Computing dynamic coefficients for time={t}")
    lse = setup_dynamic_lse(
        mpf_trotter_steps,
        t,
        identity_factory,
        exact_factory,
        approx_factory,
        mps_initial_state,
    )
    problem, coeffs = setup_frobenius_problem(lse)
    try:
        problem.solve()
        mpf_dynamic_coeffs_list.append(coeffs.value)
    except Exception as error:
        mpf_dynamic_coeffs_list.append(np.zeros(len(mpf_trotter_steps)))
        print(error, "Calculation Failed for time", t)
    print("")

Computing dynamic coefficients for time=0.5

Computing dynamic coefficients for time=0.6

Computing dynamic coefficients for time=0.7

Computing dynamic coefficients for time=0.7999999999999999

Computing dynamic coefficients for time=0.8999999999999999

Computing dynamic coefficients for time=0.9999999999999999

Computing dynamic coefficients for time=1.0999999999999999

Computing dynamic coefficients for time=1.1999999999999997

Computing dynamic coefficients for time=1.2999999999999998

Computing dynamic coefficients for time=1.4

Computing dynamic coefficients for time=1.4999999999999998



Bilang tagapagpahiwatig kung ang isang MPF na itinayo gamit ang mga coefficient na ito ay magbibigay ng magandang resulta, maaari nating gamitin ang L1-norm (tingnan din ang Ref. [\[1\]](#references)).

In [ ]:
import matplotlib.pyplot as plt

sym = {1: "^", 2: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    trotter_curve, trotter_curve_error = [], []
    for trotter_expvals, trotter_stds in zip(
        mpf_expvals_all_times, mpf_stds_all_times
    ):
        trotter_curve.append(trotter_expvals[k])
        trotter_curve_error.append(trotter_stds[k])

    plt.errorbar(
        trotter_times,
        trotter_curve,
        yerr=trotter_curve_error,
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )  # , , )

# Get expectation values at all times for the static MPF with exact coeffs
exact_mpf_curve, exact_mpf_curve_error = [], []
for trotter_expvals, trotter_stds in zip(
    mpf_expvals_all_times, mpf_stds_all_times
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(coeffs_exact.value, trotter_stds)
            ]
        )
    )
    exact_mpf_curve_error.append(mpf_std)
    exact_mpf_curve.append(trotter_expvals @ coeffs_exact.value)

plt.errorbar(
    trotter_times,
    exact_mpf_curve,
    yerr=exact_mpf_curve_error,
    markersize=4,
    marker="o",
    label="Static MPF - Exact",
    color="purple",
)


# Get expectation values at all times for the static MPF with approximate
approx_mpf_curve, approx_mpf_curve_error = [], []
for trotter_expvals, trotter_stds in zip(
    mpf_expvals_all_times, mpf_stds_all_times
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(coeffs_approx.value, trotter_stds)
            ]
        )
    )
    approx_mpf_curve_error.append(mpf_std)
    approx_mpf_curve.append(trotter_expvals @ coeffs_approx.value)

plt.errorbar(
    trotter_times,
    approx_mpf_curve,
    yerr=approx_mpf_curve_error,
    markersize=4,
    marker="o",
    color="orange",
    label="Static MPF - Approximate",
)

# # Get expectation values at all times for the dynamic MPF
dynamic_mpf_curve, dynamic_mpf_curve_error = [], []
for trotter_expvals, trotter_stds, dynamic_coeffs in zip(
    mpf_expvals_all_times, mpf_stds_all_times, mpf_dynamic_coeffs_list
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(dynamic_coeffs, trotter_stds)
            ]
        )
    )
    dynamic_mpf_curve_error.append(mpf_std)
    dynamic_mpf_curve.append(trotter_expvals @ dynamic_coeffs)

plt.errorbar(
    trotter_times,
    dynamic_mpf_curve,
    yerr=dynamic_mpf_curve_error,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)


plt.plot(
    exact_evolution_times,
    exact_expvals,
    lw=3,
    color="red",
    label="Exact time-evolution",
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) as a function of time"
)
plt.xlabel("Time")
plt.ylabel("Expectation Value")
plt.legend()
plt.grid()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/2da9c948-0.avif" alt="Output of the previous code cell" />

In the cases like the example above, where the $k=1$ PF behaves poorly at all times, the quality of the dynamic MPF results is also heavily affected. In such situations, it is useful to investigate the possibility of using individual PFs with higher number of Trotter steps to improve the overall quality of the results. In these simulations, we see the interplay of different types of errors: error from finite sampling, and Trotter error from the product formulas. MPF helps to reduce the Trotter error due to the product formulas but incur in higher sampling error compared to the product formulas. This can be advantageous, as product formulas can reduce the sampling error with increased sampling, but the systematic error due to the Trotter approximation remains untouched.

Another interesting behavior that we can observe from the plot is that the expectation value for the PF for $k=1$ starts to behave erratically (on top of not being a good approximation for the exact one) at times for which $t/k > 1 $, as explained in the [guide](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html) on how to choose the number of Trotter steps.

### Step 1: Map classical inputs to a quantum problem
Let's now consider a single time $t=1.0$ and calculate the expectation value of the magnetization with the various methods using one QPU. The particular choice of $t$ was done so to maximize the difference between the various methods and observe their relative efficacy. To determine the window of time for which dynamic MPF is guaranteed to produce observables with lower error than any of the individual Trotter formulas within the multi-product, we can implement the “MPF test” - see equation (17) and surrounding text in [\[3\]](#references).

#### Set up the Trotter circuits

At this point, we have found our expansion coefficients, $x$, and all that is left to do is to generate the Trotterized quantum circuits.
Once again, the [qiskit_addon_utils.problem_generators](/docs/api/qiskit-addon-utils/problem-generators) module comes to the rescue with a useful function to do this:

In [26]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


total_time = 1.0
mpf_circuits = []
for k in mpf_trotter_steps:
    # Initial Neel state preparation
    circuit = QuantumCircuit(L)
    circuit.x([i for i in range(L) if i % 2 != 0])

    trotter_circ = generate_time_evolution_circuit(
        hamiltonian,
        synthesis=SuzukiTrotter(order=order, reps=k),
        time=total_time,
    )

    circuit.compose(trotter_circ, inplace=True)

    mpf_circuits.append(pm.run(circuit))

In [27]:
mpf_circuits[-1].draw("mpl", fold=-1, scale=0.4)

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/87d2ac0c-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum hardware execution
Let's return to the calculation of the expectation value for a single time point. We'll pick a backend for executing the experiment on hardware.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService


service = QiskitRuntimeService()
backend = service.least_busy(min_num_qubits=127)
print(backend)

qubits = list(range(backend.num_qubits))

Tandaan po na mayroon kayong ganap na kalayaan kung paano lutasin ang optimization problem na ito, na nangangahulugang maaari ninyong baguhin ang optimization solver, ang mga convergence threshold nito, at iba pa.
Tingnan ang kaukulang gabay sa [How to use the approximate model.](https://qiskit.github.io/qiskit-addon-mpf/how_tos/using_approximate_model.html)
#### Dynamic MPF coefficients

Sa nakaraang seksyon, nagpakilala tayo ng static MPF na nagpapabuti sa karaniwang Trotter approximation. Gayunpaman, hindi kinakailangan na mino-minimize ng static na bersyon na ito ang error sa approximation. Sa konkretong paraan, ang static MPF na tinatawag na $\mu^S(t)$, ay hindi ang pinakamainam na projection ng $\rho(t)$ sa subspace na sinasaklaw ng mga product-formula state na ${\rho_{k_i}(t)}_{i=1}^r$.

Upang matugunan ito, isinasaalang-alang natin ang isang dynamic MPF (ipinakikilala sa Ref. [\[2\]](#references) at ipinakita sa eksperimento sa Ref. [\[3\]](#references)) na talagang mino-minimize ang approximation error sa Frobenius norm. Pormal na sinasabi, nakatuon tayo sa pag-minimize ng

$$
\|\rho(t) - \mu^D(t)\|_F^2 \;=\; \mathrm{Tr}\bigl[ \left( \rho(t) - \mu^D(t)\right)^2 \bigr],
$$

kaugnay ng ilang mga coefficient na $x_i(t)$ sa bawat oras na $t$. Ang *pinakamainam* na projector sa Frobenius norm ay $\mu^D(t) \;=\; \sum_{i=1}^r x_i(t)\,\rho_{k_i}(t)$, at tinatawag nating $\mu^D(t)$ ang *dynamic* MPF. Sa pamamagitan ng paglalagay ng mga kahulugan sa itaas:

$$
\|\rho(t) - \mu^D(t)\|_F^2
\;=\; \\
= \mathrm{Tr}\bigl[ \left( \rho(t) - \mu^D(t)\right)^2 \bigr]
\;=\; \\
= \mathrm{Tr}\bigl[ \left( \rho(t) - \sum_{i=1}^r x_i(t)\,\rho_{k_i}(t) \right) \left(  \rho(t) - \sum_{j=1}^r x_j(t)\,\rho_{k_j}(t) \right) \bigr]
\;=\; \\
= 1 \;+\; \sum_{i,j=1}^r M_{i,j}(t)\,x_i(t)\,x_j(t)
\;-\;
2 \sum_{i=1}^r L_i^{\mathrm{exact}}(t)\,x_i(t),
$$

kung saan ang $M_{i,j}(t)$ ay ang *Gram matrix*, na tinukoy ng

$$
M_{i,j}(t) \;=\; \mathrm{Tr}\bigl[\rho_{k_i}(t)\,\rho_{k_j}(t)\bigr]
\;=\;
\bigl|\langle \psi_{\mathrm{in}} \!\mid S\bigl(t/k_i\bigr)^{-k_i}\,S\bigl(t/k_j\bigr)^{k_j} \!\mid \psi_{\mathrm{in}} \rangle \bigr|^2.
$$

at

$$
L_i^{\mathrm{exact}}(t) = \mathrm{Tr}[\rho(t)\,\rho_{k_i}(t)]
$$

ay kumakatawan sa overlap sa pagitan ng eksaktong estado na $\rho(t)$ at ng bawat product-formula approximation na $\rho_{k_i}(t)$. Sa mga praktikal na sitwasyon, ang mga overlap na ito ay maaaring masukat lamang nang may katumpakan dahil sa ingay o limitadong access sa $\rho(t)$.

Dito, ang $\lvert\psi_{\mathrm{in}}\rangle$ ay ang paunang estado, at ang $S(\cdot)$ ay ang operasyon na inilalapat sa product formula. Sa pamamagitan ng pagpili ng mga coefficient na $x_i(t)$ na nagmi-minimize sa expression na ito (at pangangasiwa ng approximate overlap data kapag hindi ganap na kilala ang $\rho(t)$), nakakakuha tayo ng "pinakamahusay" (sa kahulugan ng Frobenius norm) na dynamic approximation ng $\rho(t)$ sa loob ng MPF subspace. Ang mga dami na $L_i(t)$ at $M_{i,j}(t)$ ay maaaring kalkulahin nang mahusay gamit ang mga pamamaraan ng tensor network [\[3\]](#references). Nagbibigay ang MPF Qiskit addon ng ilang "backend" para isagawa ang kalkulasyon. Ipinapakita ng halimbawa sa ibaba ang pinaka-flexible na paraan para gawin ito, at ang [TeNPy layer-based backend](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.tenpy_layers.html#module-qiskit_addon_mpf.backends.tenpy_layers) docs ay nagpapaliwanag din nang may detalye. Upang magamit ang pamamaraang ito, magsimula sa circuit na nagpapatupad ng nais na time-evolution at lumikha ng mga modelo na kumakatawan sa mga operasyong ito mula sa mga layer ng kaukulang circuit. Sa huli, lumilikha ng isang `Evolver` na object na maaaring gamitin para makabuo ng mga time-evolved na dami na $M_{i,j}(t)$ at $L_i(t)$. Nagsisimula tayo sa paglikha ng  `Evolver` na object na katumbas ng approximate time-evolution ([`ApproxEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ApproxEvolverFactory)) na ipinapatupad ng mga circuit. Sa partikular, bigyang-pansin ang variable na `order` upang matiyak na magkatugma ang mga ito. Pansinin na sa pagbuo ng mga circuit na katumbas ng approximate time-evolution, gumagamit tayo ng placeholder na mga halaga para sa `time = 1.0` at sa bilang ng mga Trotter step (`reps=1`). Ang tamang mga approximating circuit ay ginagawa ng dynamic problem solver sa `setup_dynamic_lse`.

In [ ]:
import copy
from qiskit.transpiler import Target, CouplingMap

target = backend.target
instruction_2q = "cz"

cmap = target.build_coupling_map(filter_idle_qubits=True)
cmap_list = list(cmap.get_edges())

max_meas_err = 0.012
min_t2 = 40
max_twoq_err = 0.005

# Remove qubits with bad measurement or t2
cust_cmap_list = copy.deepcopy(cmap_list)
for q in range(target.num_qubits):
    meas_err = target["measure"][(q,)].error
    if target.qubit_properties[q].t2 is not None:
        t2 = target.qubit_properties[q].t2 * 1e6
    else:
        t2 = 0
    if meas_err > max_meas_err or t2 < min_t2:
        # print(q)
        for q_pair in cmap_list:
            if q in q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue

# Remove qubits with bad 2q gate or t2
for q in cmap_list:
    twoq_gate_err = target[instruction_2q][q].error
    if twoq_gate_err > max_twoq_err:
        # print(q)
        for q_pair in cmap_list:
            if q == q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue


cust_cmap = CouplingMap(cust_cmap_list)

cust_target = Target.from_configuration(
    basis_gates=backend.configuration().basis_gates
    + ["measure"],  # or whatever new set of gates
    coupling_map=cust_cmap,
)

sorted_components = sorted(
    [list(comp.physical_qubits) for comp in cust_cmap.connected_components()],
    reverse=True,
)
print("size of largest component", len(sorted_components[0]))

size of largest component 10


> **Warning:** Ang mga opsyon ng `LayerwiseEvolver` na nagtatakda ng mga detalye ng tensor network simulation ay dapat piliin nang maingat upang maiwasan ang pagtatayo ng isang ill-defined na optimization problem.
Pagkatapos ay itinatayo natin ang exact evolver (halimbawa, [`ExactEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ExactEvolverFactory)), na nagbabalik ng isang [`Evolver`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.html#qiskit_addon_mpf.backends.Evolver) na object na nagkalkula ng tunay o "sanggunian" na time-evolution. Sa realidad, aapproximate natin ang eksaktong evolution sa pamamagitan ng paggamit ng higher-order Suzuki–Trotter formula o iba pang maaasahang pamamaraan na may maliit na time step. Sa ibaba, ina-approximate natin ang eksaktong time-evolved state gamit ang fourth-order Suzuki-Trotter formula na may maliit na time step na `dt=0.1`, na nangangahulugang ang bilang ng mga Trotter step na ginagamit sa oras na $t$ ay $k=t/dt$. Tinutukoy din natin ang ilang TeNPy-specific na truncation options para limitahan ang maximum bond dimension ng pinagbabatayang tensor network, pati na rin ang minimum singular values ng mga hiwalay na bond ng tensor network. Ang mga parameter na ito ay maaaring makaapekto sa katumpakan ng expectation value na kinakalkula gamit ang dynamic MPF coefficients, kaya't mahalaga na suriin ang hanay ng mga halaga upang mahanap ang pinakamainam na balanse sa pagitan ng oras ng pagkalkula at katumpakan. Tandaan na ang pagkalkula ng mga MPF coefficient ay hindi nakasalalay sa expectation value ng PF na nakuha mula sa pagpapatupad sa hardware, at samakatuwid ay maaari itong i-tune sa post-processing.

In [34]:
cust_cmap.draw()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/c5d8e90b-0.avif" alt="Output of the previous code cell" />

Susunod, likhain ang paunang estado ng iyong sistema sa format na katugma sa TeNPy (halimbawa, isang `MPS_neel_state`=$\vert 0101...01 \rangle$). Itinatayo nito ang many-body wavefunction na ie-evolve mo sa oras na $\lvert\psi_{\mathrm{in}}\rangle$ bilang isang tensor.

In [72]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

transpiler = generate_preset_pass_manager(
    optimization_level=3, target=cust_target
)

transpiled_circuits = [transpiler.run(circ) for circ in mpf_circuits]

qubits_layouts = [
    [
        idx
        for idx, qb in circuit.layout.initial_layout.get_physical_bits().items()
        if qb._register.name != "ancilla"
    ]
    for circuit in transpiled_circuits
]

transpiled_circuits = []
for circuit, layout in zip(mpf_circuits, qubits_layouts):
    transpiler = generate_preset_pass_manager(
        optimization_level=3, backend=backend, initial_layout=layout
    )
    transpiled_circuit = transpiler.run(circuit)
    transpiled_circuits.append(transpiled_circuit)


# transform the observable defined on virtual qubits to
# an observable defined on all physical qubits
isa_observables = [
    observable.apply_layout(circ.layout) for circ in transpiled_circuits
]

In [73]:
print(transpiled_circuits[-1].depth(lambda x: x.operation.num_qubits == 2))
print(transpiled_circuits[-1].count_ops())
transpiled_circuits[-1].draw("mpl", idle_wires=False, fold=False)

51
OrderedDict([('sx', 310), ('rz', 232), ('cz', 132), ('x', 19)])


<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/25ce07a6-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives
With the Estimator primitive we can obtain the estimation of expectation value from the QPU. We execute the optimized AQC circuits with additional error mitigation and suppression techniques.

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator


estimator = Estimator(mode=backend)
estimator.options.default_shots = 30000

# Set simple error suppression/mitigation options
estimator.options.dynamical_decoupling.enable = True
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = True
estimator.options.twirling.num_randomizations = "auto"
estimator.options.twirling.strategy = "active-accum"
estimator.options.resilience.measure_mitigation = True
estimator.options.experimental.execution_path = "gen3-turbo"

estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 3, 5)
estimator.options.resilience.zne.extrapolator = ("exponential", "linear")

estimator.options.environment.job_tags = ["mpf small"]


job = estimator.run(
    [
        (circ, observable)
        for circ, observable in zip(transpiled_circuits, isa_observables)
    ]
)

Sa wakas, i-plot ang mga expectation value na ito sa buong oras ng evolution.

In [39]:
result_exp = job.result()
evs_exp = [res.data.evs for res in result_exp]
evs_std = [res.data.stds for res in result_exp]

print(evs_exp)

[array(-0.06361607), array(-0.23820448), array(-0.50271805)]


![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/2da9c948-0.avif)

Sa mga kaso tulad ng halimbawa sa itaas, kung saan ang $k=1$ PF ay hindi gumaganap nang maayos sa lahat ng oras, ang kalidad ng mga resulta ng dynamic MPF ay lubos din na naaapektuhan. Sa mga ganitong sitwasyon, kapaki-pakinabang na suriin ang posibilidad ng paggamit ng mga indibidwal na PF na may mas mataas na bilang ng mga Trotter step upang mapabuti ang pangkalahatang kalidad ng mga resulta. Sa mga simulation na ito, nakikita natin ang ugnayan ng iba't ibang uri ng mga error: error mula sa limitadong sampling, at Trotter error mula sa mga product formula. Tinutulungan ng MPF na bawasan ang Trotter error dahil sa mga product formula ngunit nagdudulot ng mas mataas na sampling error kumpara sa mga product formula. Maaari itong maging kapaki-pakinabang, dahil ang mga product formula ay maaaring bawasan ang sampling error sa pamamagitan ng mas maraming sampling, ngunit ang systematic error dahil sa Trotter approximation ay nananatiling hindi nababago.

Isa pang kawili-wiling pag-uugali na maaari nating masundan mula sa plot ay ang expectation value para sa PF para sa $k=1$ ay nagsisimulang kumilos nang pabago-bago (bukod sa hindi maging magandang approximation para sa eksaktong isa) sa mga oras kung saan ang $t/k > 1 $, tulad ng ipinaliwanag sa [gabay](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html) kung paano pumili ng bilang ng mga Trotter step.
### Step 1: Map classical inputs to a quantum problem
Isaalang-alang natin ngayon ang isang solong oras na $t=1.0$ at kalkulahin ang expectation value ng magnetization gamit ang iba't ibang pamamaraan na gumagamit ng isang QPU. Ang partikular na pagpili ng $t$ ay ginawa upang mapakinabangan ang pagkakaiba sa pagitan ng iba't ibang pamamaraan at mapansin ang kaukulang bisa ng bawat isa. Upang matukoy ang window ng oras kung saan ginagarantiyahan ng dynamic MPF na makabuo ng mga observable na may mas mababang error kaysa sa alinman sa mga indibidwal na Trotter formula sa loob ng multi-product, maaari nating ipatupad ang "MPF test" — tingnan ang equation (17) at kaugnay na teksto sa [\[3\]](#references).
#### Set up the Trotter circuits

Sa puntong ito, nahanap na natin ang ating mga expansion coefficient na $x$, at ang natitira na lang ay ang pagbuo ng mga Trotterized quantum circuit.
Muli, ang module na [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) ay nakatutulong na may kapaki-pakinabang na function para dito:

In [29]:
exact_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_exact.value, evs_std)
        ]
    )
)
print(
    "Exact static MPF expectation value: ",
    evs_exp @ coeffs_exact.value,
    "+-",
    exact_mpf_std,
)
approx_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_approx.value, evs_std)
        ]
    )
)
print(
    "Approximate static MPF expectation value: ",
    evs_exp @ coeffs_approx.value,
    "+-",
    approx_mpf_std,
)
dynamic_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(mpf_dynamic_coeffs_list[7], evs_std)
        ]
    )
)
print(
    "Dynamic MPF expectation value: ",
    evs_exp @ mpf_dynamic_coeffs_list[7],
    "+-",
    dynamic_mpf_std,
)

Exact static MPF expectation value:  -0.6329590442738475 +- 0.012798249760406036
Approximate static MPF expectation value:  -0.5690390035339492 +- 0.010459559917168473
Dynamic MPF expectation value:  -0.4655579758795695 +- 0.007639139186720507


Finally, for this small problem we can compute the exact reference value using [scipy.linalg.expm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.expm.html) as follows:

In [30]:
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

exp_H = expm(-1j * total_time * hamiltonian.to_matrix())

initial_state_circuit = QuantumCircuit(L)
initial_state_circuit.x([i for i in range(L) if i % 2 != 0])
initial_state = Statevector(initial_state_circuit).data

time_evolved_state = exp_H @ initial_state

exact_obs = (
    time_evolved_state.conj() @ observable.to_matrix() @ time_evolved_state
)
print("Exact expectation value ", exact_obs.real)

Exact expectation value  -0.39909900734489434


In [31]:
sym = {1: "^", 2: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    plt.errorbar(
        k,
        evs_exp[k],
        yerr=evs_std[k],
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )  # , , )


plt.errorbar(
    3,
    evs_exp @ coeffs_exact.value,
    yerr=exact_mpf_std,
    markersize=4,
    marker="o",
    color="purple",
    label="Static MPF",
)

plt.errorbar(
    4,
    evs_exp @ coeffs_approx.value,
    yerr=approx_mpf_std,
    markersize=4,
    marker="o",
    color="orange",
    label="Approximate static MPF",
)

plt.errorbar(
    5,
    evs_exp @ mpf_dynamic_coeffs_list[7],
    yerr=dynamic_mpf_std,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)

plt.axhline(
    y=exact_obs.real,
    linestyle="--",
    color="red",
    label="Exact time-evolution",
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) at time {total_time} for the different methods "
)
plt.xlabel("Method")
plt.ylabel("Expectation Value")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=2)
plt.grid(alpha=0.1)
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/a3eefe73-0.avif" alt="Output of the previous code cell" />

Pagkatapos ay tinatanggal natin ang mga outlier qubit mula sa coupling map upang matiyak na hindi isasama ng transpiler sa layout stage ang mga ito. Sa ibaba, ginagamit natin ang iniulat na mga katangian ng backend na nakaimbak sa `target` na object at tinatanggal ang mga qubit na may measurement error o two-qubit gate na lampas sa isang tiyak na threshold (`max_meas_err`, `max_twoq_err`) o may $T_2$ time (na nagtatakda ng pagkawala ng coherence) na mas mababa sa isang tiyak na threshold (`min_t2`).

In [32]:
def relative_error(ev, exact_ev):
    return abs(ev - exact_ev)


relative_error_k = [relative_error(ev, exact_obs.real) for ev in evs_exp]
relative_error_mpf = relative_error(evs_exp @ mpf_coeffs, exact_obs.real)
relative_error_approx_mpf = relative_error(
    evs_exp @ coeffs_approx.value, exact_obs.real
)
relative_error_dynamic_mpf = relative_error(
    evs_exp @ mpf_dynamic_coeffs_list[7], exact_obs.real
)

print("relative error for each trotter steps", relative_error_k)
print("relative error with MPF exact coeffs", relative_error_mpf)
print("relative error with MPF approx coeffs", relative_error_approx_mpf)
print("relative error with MPF dynamic coeffs", relative_error_dynamic_mpf)

relative error for each trotter steps [0.33548293650112293, 0.16089452939226306, 0.10361904247828346]
relative error with MPF exact coeffs 0.2338600369291003
relative error with MPF approx coeffs 0.16993999618905486
relative error with MPF dynamic coeffs 0.06645896853467514


## Part II: scale it up

Let's scale the problem up beyond what is possible to simulate exactly. In this section we will focus on reproducing some of the results shown in Ref. [\[3\]](#references).

### Step 1: Map classical inputs to a quantum problem

#### Hamiltonian

For the large-scale example, we use the XXZ model on a line of 50 sites:

$$
\hat{\mathcal{H}}_{XXZ} = \sum_{i=1}^{L-1} J_{i,(i+1)}\left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ 2\cdot Z_i Z_{(i+1)} \right) \, ,
$$

where $J_{i,(i+1)}$ is a random coefficient corresponding to edge $(i, i+1)$. This is the Hamiltonian considered in the demonstration presented in Ref. [\[3\]](#references).

In [33]:
L = 50
# Generate some coupling map to use for this example
coupling_map = CouplingMap.from_line(L, bidirectional=False)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/34bf68ac-0.avif" alt="Output of the previous code cell" />

In [ ]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp, Pauli


# Generate random coefficients for our XXZ Hamiltonian
np.random.seed(0)
even_edges = list(coupling_map.get_edges())[::2]
odd_edges = list(coupling_map.get_edges())[1::2]

Js = np.random.uniform(0.5, 1.5, size=L)
hamiltonian = SparsePauliOp(Pauli("I" * L))
for i, edge in enumerate(even_edges + odd_edges):
    hamiltonian += SparsePauliOp.from_sparse_list(
        [
            ("XX", (edge), 2 * Js[i]),
            ("YY", (edge), 2 * Js[i]),
            ("ZZ", (edge), 4 * Js[i]),
        ],
        num_qubits=L,
    )

print(hamiltonian)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXX', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYY', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIIII', 'IIIIIIIIIIII

![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/c5d8e90b-0.avif)

Pagkatapos ay maaari nating i-map ang circuit at observable sa mga pisikal na qubit ng device.

In [35]:
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L
)
print(observable)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIIIIIIIIIIIIIIIIII'],
              coeffs=[1.+0.j])


#### Choose Trotter steps
The experiment showcased in Fig. 4 of Ref. [\[3\]](#references) uses $k_j = [2, 3, 4]$ symmetric Trotter steps of order $2$. We focus on the results for time $t=3$, where the MPF and a PF with a higher number of Trotter steps (6 in this case) have the same Trotter error. However, the MPF expectation value is calculated from circuits corresponding to the lower number of Trotter steps and thus shallower. In practice, even if the MPF and the deeper Trotter steps circuit have the same Trotter error, we expect the experimental expectation value calculated from the MPF circuits to be closer to the theory one, as it entails running shallower circuits less exposed to hardware noise compared to the circuit corresponding to the higher Trotter step PF.

In [36]:
total_time = 3
mpf_trotter_steps = [2, 3, 4]
order = 2
symmetric = True

#### Set up the LSE
Here we look at the static MPF coefficients for this problem.

In [37]:
lse = setup_static_lse(mpf_trotter_steps, order=order, symmetric=symmetric)
mpf_coeffs = lse.solve()
print(
    f"The static coefficients associated with the ansatze are: {mpf_coeffs}"
)
print("L1 norm:", np.linalg.norm(mpf_coeffs, ord=1))

The static coefficients associated with the ansatze are: [ 0.26666667 -2.31428571  3.04761905]
L1 norm: 5.628571428571431


In [38]:
model_approx, coeffs_approx = setup_sum_of_squares_problem(
    lse, max_l1_norm=2.0
)
model_approx.solve()
print(coeffs_approx.value)
print(
    "L1 norm of the approximate coefficients:",
    np.linalg.norm(coeffs_approx.value, ord=1),
)

[-0.24255546 -0.25744454  1.5       ]
L1 norm of the approximate coefficients: 2.0


### Hakbang 4: I-post-process at ibalik ang resulta sa nais na klasikal na format
Ang tanging hakbang sa post-processing ay ang pagsasama ng expected value na nakuha mula sa Qiskit Runtime primitives sa iba't ibang Trotter steps gamit ang kani-kanilang MPF coefficients. Para sa isang observable na $A$ mayroon tayong:

$$ \langle A \rangle_{\text{mpf}}  = \text{Tr} [A \mu(t)] = \sum_{j} x_j  \text{Tr} [A \rho_{k_j}] = \sum_{j} x_j \langle A \rangle_j$$

Una, kinukuha natin ang mga indibidwal na expected value na nakuha para sa bawat isa sa mga Trotter circuit:

In [ ]:
# Create approximate time-evolution circuits
single_2nd_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=order)
)
single_2nd_order_circ = pm.run(single_2nd_order_circ)  # collect XX and YY

# Find layers in the circuit
layers = slice_by_depth(single_2nd_order_circ, max_slice_depth=1)

# Create tensor network models
models = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz") for layer in layers
]

# Create the time-evolution object
approx_factory = partial(
    LayerwiseEvolver,
    layers=models,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 4,
    },
)

# Create exact time-evolution circuits
single_4th_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=4)
)
single_4th_order_circ = pm.run(single_4th_order_circ)
exact_model_layers = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz")
    for layer in slice_by_depth(single_4th_order_circ, max_slice_depth=1)
]

# Create the time-evolution object
exact_factory = partial(
    LayerwiseEvolver,
    layers=exact_model_layers,
    dt=0.1,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 3,
    },
)


def identity_factory():
    return MPOState.initialize_from_lattice(models[0].lat, conserve=True)


mps_initial_state = MPS_neel_state(models[0].lat)


lse = setup_dynamic_lse(
    mpf_trotter_steps,
    total_time,
    identity_factory,
    exact_factory,
    approx_factory,
    mps_initial_state,
)
problem, coeffs = setup_frobenius_problem(lse)
try:
    problem.solve()
    mpf_dynamic_coeffs = coeffs.value
except Exception as error:
    print(error, "Calculation Failed for time", total_time)
print("")

#### Construct each of the Trotter circuits in our MPF decomposition

In [41]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


mpf_circuits = []
for k in mpf_trotter_steps:
    # Initial state preparation |1010..>
    circuit = QuantumCircuit(L)
    circuit.x([i for i in range(L) if i % 2])

    trotter_circ = generate_time_evolution_circuit(
        hamiltonian,
        synthesis=SuzukiTrotter(reps=k, order=order),
        time=total_time,
    )

    circuit.compose(trotter_circ, qubits=range(L), inplace=True)

    mpf_circuits.append(circuit)

Susunod, pinagsasama-sama lamang natin ang mga ito kasama ang ating mga MPF coefficient upang makuha ang kabuuang expected value ng MPF. Sa ibaba, ginagawa natin ito para sa bawat isa sa mga iba't ibang paraan kung saan kinomputin natin ang $x$.

In [42]:
k = 6

# Initial state preparation |1010..>
comp_circuit = QuantumCircuit(L)
comp_circuit.x([i for i in range(L) if i % 2])


trotter_circ = generate_time_evolution_circuit(
    hamiltonian,
    synthesis=SuzukiTrotter(reps=k, order=order),
    time=total_time,
)

comp_circuit.compose(trotter_circ, qubits=range(L), inplace=True)


mpf_circuits.append(comp_circuit)

### Step 2: Optimize problem for quantum hardware execution

In [ ]:
import copy
from qiskit.transpiler import Target, CouplingMap

target = backend.target
instruction_2q = "cz"

cmap = target.build_coupling_map(filter_idle_qubits=True)
cmap_list = list(cmap.get_edges())

max_meas_err = 0.055
min_t2 = 30
max_twoq_err = 0.01

# Remove qubits with bad measurement or t2
cust_cmap_list = copy.deepcopy(cmap_list)
for q in range(target.num_qubits):
    meas_err = target["measure"][(q,)].error
    if target.qubit_properties[q].t2 is not None:
        t2 = target.qubit_properties[q].t2 * 1e6
    else:
        t2 = 0
    if meas_err > max_meas_err or t2 < min_t2:
        # print(q)
        for q_pair in cmap_list:
            if q in q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue

# Remove qubits with bad 2q gate or t2
for q in cmap_list:
    twoq_gate_err = target[instruction_2q][q].error
    if twoq_gate_err > max_twoq_err:
        # print(q)
        for q_pair in cmap_list:
            if q == q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue


cust_cmap = CouplingMap(cust_cmap_list)

cust_target = Target.from_configuration(
    basis_gates=backend.configuration().basis_gates
    + ["measure"],  # or whatever new set of gates
    coupling_map=cust_cmap,
)

sorted_components = sorted(
    [list(comp.physical_qubits) for comp in cust_cmap.connected_components()],
    reverse=True,
)
print("size of largest component", len(sorted_components[0]))

size of largest component 73


In [104]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

transpiler = generate_preset_pass_manager(
    optimization_level=3, target=cust_target
)

transpiled_circuits = [transpiler.run(circ) for circ in mpf_circuits]

qubits_layouts = [
    [
        idx
        for idx, qb in circuit.layout.initial_layout.get_physical_bits().items()
        if qb._register.name != "ancilla"
    ]
    for circuit in transpiled_circuits
]

transpiled_circuits = []
for circuit, layout in zip(mpf_circuits, qubits_layouts):
    transpiler = generate_preset_pass_manager(
        optimization_level=3, backend=backend, initial_layout=layout
    )
    transpiled_circuit = transpiler.run(circuit)
    transpiled_circuits.append(transpiled_circuit)


# transform the observable defined on virtual qubits to
# an observable defined on all physical qubits
isa_observables = [
    observable.apply_layout(circ.layout) for circ in transpiled_circuits
]

### Step 3: Execute using Qiskit primitives

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(mode=backend)
estimator.options.default_shots = 30000

# Set simple error suppression/mitigation options
estimator.options.dynamical_decoupling.enable = True
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = True
estimator.options.twirling.num_randomizations = "auto"
estimator.options.twirling.strategy = "active-accum"
estimator.options.resilience.measure_mitigation = True
estimator.options.experimental.execution_path = "gen3-turbo"

estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 1.2, 1.4)
estimator.options.resilience.zne.extrapolator = "linear"

estimator.options.environment.job_tags = ["mpf large"]

job_50 = estimator.run(
    [
        (circ, observable)
        for circ, observable in zip(transpiled_circuits, isa_observables)
    ]
)

### Step 4: Post-process and return result in desired classical format

In [46]:
result = job_50.result()
evs = [res.data.evs for res in result]
std = [res.data.stds for res in result]

print(evs)
print(std)

[array(-0.08034071), array(-0.00605026), array(-0.15345759), array(-0.18127293)]
[array(0.04482517), array(0.03438413), array(0.21540776), array(0.21520829)]


In [52]:
exact_mpf_std = np.sqrt(
    sum([(coeff**2) * (std**2) for coeff, std in zip(mpf_coeffs, std[:3])])
)
print(
    "Exact static MPF expectation value: ",
    evs[:3] @ mpf_coeffs,
    "+-",
    exact_mpf_std,
)
approx_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_approx.value, std[:3])
        ]
    )
)
print(
    "Approximate static MPF expectation value: ",
    evs[:3] @ coeffs_approx.value,
    "+-",
    approx_mpf_std,
)
dynamic_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(mpf_dynamic_coeffs, std[:3])
        ]
    )
)
print(
    "Dynamic MPF expectation value: ",
    evs[:3] @ mpf_dynamic_coeffs,
    "+-",
    dynamic_mpf_std,
)

Exact static MPF expectation value:  -0.47510243192011536 +- 0.6613940032465087
Approximate static MPF expectation value:  -0.20914170384216998 +- 0.32341567460419135
Dynamic MPF expectation value:  -0.07994951978722761 +- 0.07423091963310202


In [ ]:
sym = {2: "^", 3: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    plt.errorbar(
        k,
        evs[k],
        yerr=std[k],
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )


plt.errorbar(
    3,
    evs[-1],
    yerr=std[-1],
    alpha=0.5,
    markersize=8,
    marker="x",
    color="blue",
    label="6 Trotter steps",
)


plt.errorbar(
    4,
    evs[:3] @ mpf_coeffs,
    yerr=exact_mpf_std,
    markersize=4,
    marker="o",
    color="purple",
    label="Static MPF",
)

plt.errorbar(
    5,
    evs[:3] @ coeffs_approx.value,
    yerr=approx_mpf_std,
    markersize=4,
    marker="o",
    color="orange",
    label="Approximate static MPF",
)

plt.errorbar(
    6,
    evs[:3] @ mpf_dynamic_coeffs,
    yerr=dynamic_mpf_std,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)

exact_obs = -0.24384471447172074  # Calculated via Tensor Network calculation
plt.axhline(
    y=exact_obs, linestyle="--", color="red", label="Exact time-evolution"
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) at time {total_time} for the different methods "
)
plt.xlabel("Method")
plt.ylabel("Expectation Value")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=2)
plt.grid(alpha=0.1)
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/d751af7c-0.avif" alt="Output of the previous code cell" />

## Bahagi II: palawakin ito
Palawakin natin ang problema nang higit pa sa kung ano ang posibleng ma-simulate nang eksakto. Sa seksyong ito, magtutuon tayo sa pagpapareplika ng ilang mga resulta na ipinapakita sa Ref. [\[3\]](#references).
### Hakbang 1: I-map ang mga klasikal na input sa isang quantum na problema
#### Hamiltonian

Para sa malawak na halimbawa, ginagamit natin ang XXZ model sa isang linya ng 50 sites:

$$
\hat{\mathcal{H}}_{XXZ} = \sum_{i=1}^{L-1} J_{i,(i+1)}\left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ 2\cdot Z_i Z_{(i+1)} \right) \, ,
$$

kung saan ang $J_{i,(i+1)}$ ay isang random coefficient na naaayon sa edge $(i, i+1)$. Ito ang Hamiltonian na isinasaalang-alang sa demonstrasyon na inilalahad sa Ref. [\[3\]](#references).